In [ ]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
#!pip install --upgrade pip
#!pip install imageio_ffmpeg
#!pip install moviepy
from moviepy.editor import VideoFileClip
import math

In [ ]:
def caliberation(path, file, nx, ny, size):
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
    objpoints = [] 
    imgpoints = []
    
    images = glob.glob('./'+path+'/'+file+'*'+'.jpg')
    
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, size, None, None)
    
    return mtx, dist



def undistort(image, mtx, dist):
    image = cv2.undistort(image, mtx, dist, None, mtx)
    return image



mtx, dist = caliberation('camera_cal', 'calibration', 9, 6, (720, 1280))
checker_dist = mpimg.imread("./camera_cal/calibration2.jpg")
checker_undist = undistort(checker_dist, mtx, dist)



f, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(12, 18))
ax1.imshow(checker_dist)
ax1.set_title('Original', fontsize=15)
ax2.imshow(checker_undist)
ax2.set_title('Undistorted', fontsize=15)

In [ ]:

def abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh_min=0, thresh_max = 255):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel))
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    grad_binary = np.zeros_like(scaled_sobel)
    
    grad_binary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    
    return grad_binary



In [ ]:
def mag_thresh(img, sobel_kernel=3, mag_thresh=(30, 100)):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    
    return mag_binary



   
    

In [ ]:

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    
    return dir_binary


    
    


In [ ]:

def hls_thresh(img, thresh=(100, 255)):
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output




In [ ]:
def  combine_threshold_gradient(image):
    
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh_min=50, thresh_max = 255)
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=3, thresh_min=50, thresh_max = 255)
    mag_binary = mag_thresh(image, sobel_kernel=3, mag_thresh=(50, 255))
    dir_binary = dir_threshold(image, sobel_kernel=9, thresh=(0, np.pi/2))
    hls_bin = hls_thresh(image, thresh=(170, 255))
    
    
    
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))| hls_bin == 1] = 1
    
    return combined


    

In [ ]:


def channel_Isolate(image,channel):
    ## Takes in only RBG images
    if (channel == 'R'):
        return image[:,:,0]
    
    elif (channel == 'G'):
        return image[:,:,1]
    
    elif (channel == 'B'):
        return image[:,:,2]
    
    elif (channel == 'H'):
        HSV = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        return HSV[:,:,0]
    
    elif (channel == 'S'):
        HSV = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        return HSV[:,:,1]
        
    elif (channel == 'V'):
        HSV = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        return HSV[:,:,2]
        
    elif (channel == 'L'):
        HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        return HLS[:,:,1]
    
    elif (channel == 'Cb'):
        YCrCb = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        return YCrCb[:,:,2]
    
    elif (channel == 'U'):
        LUV = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
        return LUV[:,:,2]
    
    else:
        raise Error("Channel must be either R, G, B, H, S, V, L, Cb, U")
    
def threshold_Channel(channel,thresh):
    retval, binary = cv2.threshold(channel.astype('uint8'), thresh[0], thresh[1], cv2.THRESH_BINARY)
    return binary



In [ ]:
def transform(undist,src,dst,img_size):
        
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(undist, M, img_size)
    
    return warped
    

In [ ]:

def find_lines_pixels(img):
    
    right_side_x = []
    right_side_y = []
    left_side_x = []
    left_side_y = []  
    
    past_cord = 0
    
    # right side
    for i in reversed(range(10,100)):
        histogram = np.sum(img[i*img.shape[0]/100:(i+1)*img.shape[0]/100,img.shape[1]/2:], axis=0) 
        xcord = int(np.argmax(histogram)) + 640
        ycord = int(i*img.shape[0]/100)
        if (i == 50):
            right_lane_dp = (xcord)
        if (ycord == 0 or xcord == 0):
            pass
        elif (abs(xcord-past_cord) > 100 and not(i == 99) and not( past_cord == 0)):
            pass
        elif (xcord == 640):
            pass
        else:
            right_side_x.append(xcord)
            right_side_y.append(ycord)
            past_cord = xcord

    past_cord = 0
    # left side
    for i in reversed(range(10,100)):
        histogram = np.sum(img[i*img.shape[0]/100:(i+1)*img.shape[0]/100,:img.shape[1]/2], axis=0)
        xcord = int(np.argmax(histogram))
        ycord = int(i*img.shape[0]/100)
        if (i == 50):
            left_lane_dp = (xcord)
        if (ycord == 0 or xcord == 0):
            pass
        elif (abs(xcord-past_cord) > 100 and not(i == 99) and not(past_cord == 0)):
            pass
        else:
            left_side_x.append(xcord)
            left_side_y.append(ycord)
            past_cord = xcord
    
    left_line =  (left_side_x,left_side_y)
    right_line = (right_side_x,right_side_y)
    
    left_line =  (left_line[0][1:(len(left_line[0])-1)],left_line[1][1:(len(left_line[1])-1)])
    right_line = (right_line[0][1:(len(right_line[0])-1)],right_line[1][1:(len(right_line[1])-1)])
    
    lane_middle = int((right_lane_dp - left_lane_dp)/2.)+left_lane_dp
    
    if (lane_middle-640 > 0):
        leng = 3.66/2
        mag = ((lane_middle-640)/640.*leng)
        head = ("Right",mag)
    else:
        leng = 3.66/2.
        mag = ((lane_middle-640)/640.*leng)*-1
        head = ("Left",mag)

    
    return left_line, right_line, head


In [ ]:
def lane_curve(left_line,right_line):
    
    degree_fit = 2
    
    fit_left = np.polyfit(left_line[1], left_line[0], degree_fit)
    
    fit_right = np.polyfit(right_line[1], right_line[0], degree_fit)
    
    x = [x * (3.7/700.) for x in left_line[0]]
    y = [x * (30/720.) for x in left_line[1]]
    
    curve = np.polyfit(y, x, degree_fit)
           
    return fit_left, fit_right, curve

In [ ]:
def lane_Area(undist, fit_left, fit_right, trans, src, dst, img_size, curve):
    
    global left_points,right_points
    
    left_points  = []
    right_points = []

    left = np.poly1d(fit_left)
    right = np.poly1d(fit_right)

    rad_curve = ((1 + (2*curve[0]*710/2 + curve[1])**2)**1.5)/np.absolute(2*curve[0])  
 
    for i in range(100,710,2):
        if (int(left(i)<0)):
            pass
        else:
            left_points.append([int(left(i)),i])

    for i in range(100,710,2):
        if (int(right(i)<0)):
            pass
        else:
            right_points.append([int(right(i)),i])

    
    polygon_points = right_points + list(reversed(left_points))
    polygon_points = np.array(polygon_points)

    
    overlay = np.zeros_like(trans)
    trans_image = cv2.fillPoly(overlay, [polygon_points], (0,255,0) )
    
    M = cv2.getPerspectiveTransform(dst, src)
    unwarped = cv2.warpPerspective(trans_image, M, img_size)
    
    area = cv2.contourArea(polygon_points)
    
    return unwarped, area, rad_curve, polygon_points

In [ ]:
def pipeline(image,mtx, dist):
    global polygon, area, left, right, last, frame_count, Head_b, rad_curve_b, vehicle_count, IMAGE
    
        
    img_size = (image.shape[1], image.shape[0])
    src =  np.float32([[250,700],[1200,700],[550,450],[750,450]])
    dst = np.float32([[250,700],[1200,700],[300,50],[1000,50]])
    
    
    undist = undistort(image, mtx, dist)
    trans = transform(undist,src,dst,img_size)
    
    red_threshed = threshold_Channel(channel_Isolate(trans,'R'),(220,255))
    V_threshed = threshold_Channel(channel_Isolate(trans,'V'),(220,255))
    HSV = cv2.cvtColor(trans, cv2.COLOR_RGB2HSV)
    yellow = cv2.inRange(HSV, (20, 100, 100), (50, 255, 255))
    sensitivity_1 = 68
    white = cv2.inRange(HSV, (0,0,255-sensitivity_1), (255,20,255))
    sensitivity_2 = 60
    HSL = cv2.cvtColor(trans, cv2.COLOR_RGB2HLS)
    white_2 = cv2.inRange(HSL, (0,255-sensitivity_2,0), (255,255,sensitivity_2))
    white_3 = cv2.inRange(trans, (200,200,200), (255,255,255))
    bit_image = red_threshed | V_threshed | yellow | white | white_2 | white_3
    
    #bit_image =combine_threshold_gradient(trans)
    
    left_line, right_line, head = find_lines_pixels(bit_image)
    
    fit_left, fit_right, curve = lane_curve(left_line,right_line)
    
    unwarped, area, rad_curve, polygon_points = lane_Area(undist,fit_left, fit_right, trans, src, dst, img_size, curve)
    
    a = polygon_points
    b = polygon_points
    
    ret = cv2.matchShapes(a, b, 1, 0.0)
    original = undist.copy()
    
    polygon = unwarped
    Head_b = head
        
    result = cv2.addWeighted(undist, 1, unwarped, 0.3, 0)
    
    original[475:720] = result[475:720]
    
    font = cv2.FONT_HERSHEY_DUPLEX
    
    text = Head_b[0]
    cv2.putText(original,text , (256,71), font, 1, (0,0,0), 1, cv2.LINE_AA)
    dist = math.ceil(Head_b[1]*100)/100
    cv2.putText(original,str(dist) , (181,71), font, 1, (0,0,0), 1, cv2.LINE_AA)
    rad_curve = math.ceil(rad_curve*100)/100
    cv2.putText(original,str(rad_curve) , (244,95), font, 1, (0,0,0), 1, cv2.LINE_AA)
    
    return original



In [ ]:
# Parameters
directory = 'camera_cal'
filename = 'calibration'
nx = 9 
ny = 6
cal_img_size = (720, 1280)


mtx, dist = caliberation(directory, filename, nx, ny, cal_img_size)

In [ ]:

image_1 = mpimg.imread("./test_images/test1.jpg")
Image_1 = pipeline(image_1, mtx, dist)
image_2 = mpimg.imread("./test_images/test2.jpg")
Image_2 = pipeline(image_2, mtx, dist)
image_3 = mpimg.imread("./test_images/test3.jpg")
Image_3 = pipeline(image_3, mtx, dist)
image_4 = mpimg.imread("./test_images/test4.jpg")
Image_4 = pipeline(image_4, mtx, dist)
image_5 = mpimg.imread("./test_images/test5.jpg")
Image_5 = pipeline(image_5, mtx, dist)
image_6 = mpimg.imread("./test_images/test6.jpg")
Image_6 = pipeline(image_6, mtx, dist)

mpimg.imsave("./output_images/Itest1.jpg", Image_1)
mpimg.imsave("./output_images/Itest2.jpg", Image_1)
mpimg.imsave("./output_images/Itest3.jpg", Image_1)
mpimg.imsave("./output_images/Itest4.jpg", Image_1)
mpimg.imsave("./output_images/Itest5.jpg", Image_1)
mpimg.imsave("./output_images/Itest6.jpg", Image_1)

In [ ]:
output = 'challenge_video.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
clip = clip1.fl_image(lambda img: pipeline(img, mtx, dist, dash))